In [1]:
import requests, json
from pprint import pprint
import pandas as pd

In [23]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'chips': [...],
  'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_config': {...},
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 11124519}


In [3]:
# get player data from 'elements' field
players = r['elements']

# show data for first player
pprint(players[0])

{'assists': 0,
 'bonus': 0,
 'bps': 0,
 'can_select': False,
 'can_transact': True,
 'chance_of_playing_next_round': 0,
 'chance_of_playing_this_round': 0,
 'clean_sheets': 0,
 'clean_sheets_per_90': 0,
 'code': 438098,
 'corners_and_indirect_freekicks_order': None,
 'corners_and_indirect_freekicks_text': '',
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': -1,
 'cost_change_start_fall': 1,
 'creativity': '0.0',
 'creativity_rank': 700,
 'creativity_rank_type': 315,
 'direct_freekicks_order': None,
 'direct_freekicks_text': '',
 'dreamteam_count': 0,
 'element_type': 3,
 'ep_next': '0.0',
 'ep_this': '0.0',
 'event_points': 0,
 'expected_assists': '0.00',
 'expected_assists_per_90': 0,
 'expected_goal_involvements': '0.00',
 'expected_goal_involvements_per_90': 0,
 'expected_goals': '0.00',
 'expected_goals_conceded': '0.00',
 'expected_goals_conceded_per_90': 0,
 'expected_goals_per_90': 0,
 'first_name': 'Fábio',
 'form': '0.0',
 'form_rank': 690,
 'form_r

In [4]:

import pandas as pd
pd.set_option('display.max_columns', None)

In [5]:

# create players dataframe
players = pd.json_normalize(r['elements'])

# show some information about first five players
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,1,Fábio Vieira,1,3
1,2,G.Jesus,1,4
2,3,Gabriel,1,2
3,4,Havertz,1,4
4,5,Hein,1,1


In [7]:
# create teams dataframe
teams = pd.json_normalize(r['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1250,1360,1260,1370,1240,1350,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1110,1240,1080,1160,1140,1320,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,3,None,False,0,1140,1175,1080,1140,1190,1210,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1080,1225,1070,1100,1090,1350,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1110,1150,1080,1100,1140,1200,131


In [8]:

# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],73
1,2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],240
2,3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],320
3,4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],78


In [9]:

# join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

# show joined result
df[['first_name', 'second_name', 'name']].head()

,first_name,second_name,name
0,Fábio,Ferreira Vieira,Arsenal
1,Gabriel,Fernando de Jesus,Arsenal
2,Gabriel,dos Santos Magalhães,Arsenal
3,Kai,Havertz,Arsenal
4,Karl,Hein,Arsenal


In [10]:

# join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

# show result
df[
    ['first_name', 'second_name', 'team_name', 'position_name']
].head()

,first_name,second_name,team_name,position_name
0,Fábio,Ferreira Vieira,Arsenal,Midfielder
1,Gabriel,Fernando de Jesus,Arsenal,Forward
2,Gabriel,dos Santos Magalhães,Arsenal,Defender
3,Kai,Havertz,Arsenal,Forward
4,Karl,Hein,Arsenal,Goalkeeper


In [11]:

# get data from 'element-summary/{PID}/' endpoint for PID=4
r = requests.get(base_url + 'element-summary/4/').json()

# show top-level fields for player summary
pprint(r, depth=1)

{'fixtures': [...], 'history': [...], 'history_past': [...]}


In [12]:
# show data for first gameweek
pprint(r['history'][0])

{'assists': 1,
 'bonus': 3,
 'bps': 48,
 'clean_sheets': 1,
 'creativity': '24.1',
 'element': 4,
 'expected_assists': '0.04',
 'expected_goal_involvements': '0.49',
 'expected_goals': '0.45',
 'expected_goals_conceded': '0.47',
 'fixture': 2,
 'goals_conceded': 0,
 'goals_scored': 1,
 'ict_index': '12.5',
 'influence': '54.8',
 'kickoff_time': '2024-08-17T14:00:00Z',
 'minutes': 90,
 'modified': False,
 'opponent_team': 20,
 'own_goals': 0,
 'penalties_missed': 0,
 'penalties_saved': 0,
 'red_cards': 0,
 'round': 1,
 'saves': 0,
 'selected': 1087445,
 'starts': 1,
 'team_a_score': 0,
 'team_h_score': 2,
 'threat': '46.0',
 'total_points': 12,
 'transfers_balance': 0,
 'transfers_in': 0,
 'transfers_out': 0,
 'value': 80,
 'was_home': True,
 'yellow_cards': 0}


In [13]:
def get_gameweek_history(player_id):
    '''get all gameweek info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])
    
    return df

In [14]:
# show player #4's gameweek history
get_gameweek_history(4)[
    [
        'round',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head()

,round,total_points,minutes,goals_scored,assists
0,1,12,90,1,1
1,2,2,90,0,0
2,3,8,90,1,0
3,4,2,90,0,0
4,5,2,90,0,0


In [15]:
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    
    return df


# show player #1's gameweek history
get_season_history(1)[
    [
        'season_name',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head(10)

,season_name,total_points,minutes,goals_scored,assists
0,2022/23,40,500,1,2
1,2023/24,24,290,1,3


In [16]:
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()

,id_player,first_name,second_name,web_name,name,singular_name_short
0,1,Fábio,Ferreira Vieira,Fábio Vieira,Arsenal,MID
1,2,Gabriel,Fernando de Jesus,G.Jesus,Arsenal,FWD
2,3,Gabriel,dos Santos Magalhães,Gabriel,Arsenal,DEF
3,4,Kai,Havertz,Havertz,Arsenal,FWD
4,5,Karl,Hein,Hein,Arsenal,GKP


In [18]:
from tqdm.auto import tqdm
tqdm.pandas()

d:\A - Stuff\Documents\A - Coding\Python\Prem Table\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:

# get gameweek histories for each player
points = players['id_player'].progress_apply(get_gameweek_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# join web_name
points = players[['id_player', 'web_name']].merge(
    points,
    left_on='id_player',
    right_on='element'
)

100%|██████████| 711/711 [00:52<00:00, 13.49it/s]


In [20]:
# get top scoring players
points.groupby(
    ['element', 'web_name']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head()

,element,web_name,total_points,goals_scored,assists
327,328,M.Salah,209,18,13
181,182,Palmer,153,13,7
98,99,Mbeumo,136,13,3
350,351,Haaland,123,16,1
400,401,Isak,119,13,4
